![](img/575_banner.png)

# Lecture 8: Self-attention and transformers

UBC Master of Data Science program, 2021-22

Instructor: Varada Kolhatkar

> [Attention is all you need!](https://arxiv.org/pdf/1706.03762.pdf)

## Lecture plan, imports, LO

### Lecture plan 

- iClicker questions
- Recap and limitations of LSTMs 
- Self-attention layers
- Transformer blocks 
- Break
- iClicker questions
- Multihead attention
- Transfer learning 
- Course conclusion

<br><br>

## Imports

In [1]:
import sys
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

pd.set_option("display.max_colwidth", 0)

<br><br>

### Learning outcomes

From this lecture you will be able to 

- Broadly explain the limitations of LSTMs. 
- Explain the idea of self-attention. 
- Describe the three core operations in self-attention. 
- Explain the query, key, and value roles in self-attention. 
- Explain the role of linear projections for query, key, and value in self-attention. 
- Explain transformer blocks. 
- Explain the advantages of using transformers over LSTMs. 
- Broadly explain the idea of multihead attention. 
- Broadly explain the idea of transfer learning. 

<br><br>

### Attributions

This material is heavily based on [Jurafsky and Martin, Chapter 9]((https://web.stanford.edu/~jurafsky/slp3/9.pdf)).

<br><br><br><br>

## Recap

### RNNs vs. LSTMs 

- RNNs suffer from the problem of vanishing gradients. 
    - Vanishing gradients lead to difficulty in training.  
- We saw that LSTMs mitigate this problem by introducing a cell state and managing the context better. 
- They are better suited for sequence modeling and capturing long-distance dependencies. 

![](img/RNN_alternative_representation.png)
![](img/LSTM0.png)

<!-- <img src="img/RNN_alternative_representation.png" height="400" width="400">  -->

<!-- <img src="img/LSTM0.png" height="400" width="400">  -->
<!-- </center>     -->

(Credit: [Colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

There are tons of applications of LSTMs. 

### Wiki page on LSTM applications 

![](img/lstm_applications_wiki.png)

<!-- <img src="img/lstm_applications_wiki.png" width="1500" height="1500"> -->

(Credit: [Learning to execute](https://arxiv.org/pdf/1410.4615.pdf))

### LSTM for image captioning 

![](img/RNN_LSTM_image_captioning.png)

<!-- <center> -->
<!-- <img src="img/RNN_LSTM_image_captioning.png" height="2000" width="2000">  -->
<!-- </center>     -->


(Credit: [LSTMs for image captioning](https://arxiv.org/pdf/1411.4555.pdf))

<br><br>

### music2dance

- [Dance with Melody: An LSTM-autoencoder Approach to Music-oriented Dance Synthesis](https://hcsi.cs.tsinghua.edu.cn/Paper/Paper18/MM18-TANGTAORAN.pdf)
- [Follow the Music: Dance Motion Synthesis Corresponding to Arbitrary Music](https://www.cs.ubc.ca/~rozentil/data/music2dance.pdf) (UBC researchers)

![](img/music2dance.png)

<!-- <center> -->
<!-- <img src="img/music2dance.png" height="1500" width="1500">  -->
<!-- </center>     -->


<br><br><br><br>

## ❓❓ Questions for you

iClicker cloud join link: https://join.iclicker.com/4QVT4

### Exercise 8.1: Select all of the following statements which are **True** (iClicker)

- (A) Input to an RNN or LSTM is a 3-D tensor with three components: sequence length, batch size, and input size.
- (B) In the forward pass of LSTMs, in addition to the previous hidden state we also pass the previous cell state.
- (C) In LSTMs, creating batches with sequences of similar lengths might minimize padding.
- (D) HMMs can handle variable length sequences but LSTMs cannot.   
- (E) It's almost always a good idea to use LSTMs to model text data over bag-of-words models, as they encode sequential information and are likely to beat bag-of-words models. 

<br><br><br><br>

```{admonition} Exercise 8.1: V's Solutions!
:class: tip, dropdown
- (A) True
- (B) True
- (C) True
- (D) False
- (E) False
```

### Exercise 8.2: Questions for discussion

1. Suppose you are using pre-trained embeddings to train a sentiment analysis model with LSTMs on a small corpus shown below. Now given a test sentence "amazing", would you expect the model to classify the sentence correctly? Why or why not? State your assumptions.  

| text        | sentiment           |
| ------------- |:-------------:|
| what a wonderful movie ! | positive|
| boring ... fell asleep a couple of times | negative |
| astonishing ! ! | positive |

2. In text generation using LSTMs or RNNs, why it might be a good idea to sample from the distribution rather than picking the next word with highest probability? 

```{admonition} Exercise 8.2: V's Solutions!
:class: tip, dropdown
1.  Most likely yes. Although the word **amazing** does not occur in the training corpus, we would be incorporating the information in the model, via pre-trained word embeddings, that the word amazing is similar to wonderful and astonishing which are associated with positive sentiment.
2. To add novelty.
```

<br><br><br><br>

## Self-attention networks: Transformers

### Limitations of LSTMs

- LSTMs are better than vanilla RNNs in mitigating the problem of loss of distant information caused by vanishing gradients. 
- But the underlying problem remains.
- For long sequences, there is still a loss of relevant information and difficulties in training. 
- Also, inherently sequential nature of LSTMs make them hard to parallelize. So they are slow to train. 
- This led to development of **transformers**. 
    - They are much faster to train compared to LSTMs and much better at capturing long distance dependencies. 
    - They are at the core of all state-of-the-art NLP models (e.g., BERT, GPT2, GPT3). 

### Transformers 

- Transformers provide an approach to sequence processing but they eliminate recurrent connections in RNNs and LSTMs.   
- Similar to RNNs or LSTMs, they map sequences of input vectors $(x_1, \dots, x_n)$ to sequences of output vectors $(y_1, \dots, y_n)$ of the same length.  
- They are made up of **transformer blocks** which are multilayer networks made by combining 
    - simple linear layers,
    - feedforward layers, and 
    - **self-attention layers**, which is the key innovation of transformers. 
- You will see these transformer blocks in modern language model architectures. 
- Let's first focus on self-attention layer. 
- Later we'll see how it fits in the larger transformer blocks. 

### Self-attention layer

- Self-attention allows a network to directly extract and use information from arbitrarily large contexts without the need to pass it through intermediate recurrent connections as in RNNs. 
- Below is a single backward looking self-attention layer which maps sequences of input vectors $(x_1, \dots, x_n)$ to sequences of output vectors $(y_1, \dots, y_n)$ of the same length. 
- When processing an item at time $t$, the model has access to all of the inputs up to and including the one under consideration. 
- It does not have access to the input beyond the current one. 
- Note that unlike RNNs or LSTMs, each computation can be done independently; it does not depend upon the previous computation which allows us to easily parallelize forward pass and the training of such models. 

![](img/self_attention.png)
<!-- <img src="img/self_attention.png" width="600" height="600"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Self-attention: Core idea 

- For each token in the sequence, we assign a weight based on how relevant they are to the token under consideration. 
- Calculate the output for the current token based on these weights. 

### Calculating the output $y$ for the token _nuts_ in the given context

![](img/self_attention_ex_nuts.png)

<!-- <img src="img/self_attention_ex_nuts.png" width="600" height="600"> -->

### The key operations in self-attention

So in order to calculate the output $y_i$

- We score token $x_i$ with all previous tokens $x_j$ by taking the dot product between them. 
$$\text{score}(x_i, x_j) = x_i \cdot x_j$$

- We apply $\text{softmax}$ on these scores to get probability distribution over these scores. 
$$\alpha_{ij} = \text{softmax}(\text{score}(x_i \cdot x_j)), \forall j \leq i$$

- The output is the weighted sum of the inputs seen so far, where the weights correspond to the $\alpha$ values calculated above. 
 $$y_i = \sum_{j \leq i} \alpha_{ij}x_j$$
 
These three operations represent the core of an attention-based approach. These operations can be carried out independently for each input allowing easy parallelism. 

### Query, Key, and Value roles

Note that in the process of calculating outputs corresponding to each input, each input embedding plays three kinds of roles. 

- **Query**: _the current focus of attention_ when being compared to all previous inputs. 
- **Key**: _a preceding input_ being compared to the current focus of attention.    
- **Value**: used to compute the output for the current focus of attention. 

For these three roles transformer introduces three weight matrices: $W^Q, W^K, W^V$. These weights will be used to project each input vector $x_i$ into its role as a key, query, or value.

$$q_i = W^Qx_i$$
$$k_i = W^Kx_i$$
$$v_i = W^Vx_i$$

For now let's assume that all these weight matrices have the same dimensionality and so the projected vectors in each case are going to be of the same size. 

With these projections our equations become: 

- We score the $x_i$ with all previous tokens $x_j$ by taking the dot product between $x_i$'s query vector $q_i$ and $x_j$'s key vector $k_j$:  
$$\text{score}(x_i, x_j) = q_i \cdot k_j$$

- The softmax calculation remains the same but the output calculation for $y_i$ is now based on a weighted sum over the projected vectors $v$:
 $$y_i = \sum_{j \leq i} \alpha_{ij}v_j$$
 

### Self-attention: Calculating the value of $y_3$

![](img/self_attention_ex.png)

<!-- <img src="img/self_attention_ex.png" width="400" height="400"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Scaling the dot products

- The result of a dot product can be arbitrarily large and exponentiating such values can lead to numerical issues and problems during training. 
- So the dot products are usually scaled before applying the softmax. 
- The most common scaling is where we divide the dot product by the square root of the dimensionality of the query and the key vectors. 
$$\text{score}(x_i, x_j) = \frac{q_i \cdot k_j}{\sqrt{d_k}}$$


- This is how we calculate a single output of a single time step $i$. 
- Would the output calculation at different time steps be dependent upon each other? 

### Efficient calculations with matrix multiplication 

- $X_{N \times d} \rightarrow$ matrix of all tokens in a sequence of length $N$ with each token represented with a $d$ dimensional embedding. Each row of $X$ is embedding representation of one token of the input. The we can calculate $Q, K, V$ as follows.

$$Q = XW^Q$$
$$K = XW^K$$
$$V = XW^V$$

- With these, we can now calculate all the query-key scores simultaneously as $Q \times K$. 

![](img/self_attention_calc_all.png)

<!-- <img src="img/self_attention_calc_all.png" width="300" height="300"> -->

- We can them apply softmax on all rows and multiply the resulting matrix by $V$.

$$SelfAttention(Q, K, V) = \text{softmax}(\frac{QK}{\sqrt{d_k}})V$$

- Finally, we get output sequence $y_1, \dots, y_n$.   


- What's the problem with the approach above?
    - This process goes a bit too far since the calculation of the comparisons in $QK$ results in a score for each value to each key value, _including those that follow the query_. 
    - Is this appropriate in the setting of language modeling? 

![](img/self_attention_calc_partial.png)

<!-- <img src="img/self_attention_calc_partial.png" width="300" height="300"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Transformer blocks

- In many advanced architectures, you will see transformer blocks which consists of
    - The self-attention layer
    - Additional feedforward layers
    - Residual connections
    - Normalizing layers

![](img/transformer_block.png)

<!-- <img src="img/transformer_block.png" width="350" height="350"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

- The input and output dimensions of these layers are matched so that they can be stacked. 
- In deep networks, **residual connections** are connections that pass information from a lower layer to a higher layer without going through the intermediate layer. Why? It has been shown that allowing information from the activation going forward and the gradient going backwards to skip a layer improves learning and gives higher level layers direct access to information from lawer layers. 
- We then have a summed vector (projected output of the attention or feedforward layer + input of the attention or feedforward layers). 
- **Layer normalization or layer norm** normalizes the resulting vector which improves training performance in deep neural networks keeping the values of a hidden layer in a range that facilitates gradient-based training. Layer norm applies something similar to `StandardScaler` so that the mean is 0 and standard deviation is 1 in the vector. 

<br><br><br><br>

### Break

![](img/eva-coffee.png)

<br><br><br><br>

## ❓❓ Questions for you

iClicker cloud join link: https://join.iclicker.com/4QVT4

### Exercise 8.2: Select all of the following statements which are **True** (iClicker)

- (A) The main difference between the RNN layer and a self-attention layer is that 
in self-attention, we pass the information without intermediate recurrent connections. 
- (B) In self-attention, the output $y_i$ of input $x_i$ at time $i$ is a single number. 
- (C) Calculating attention weights is quadratic in the length of the input 
since we need to compute dot products between each pair of tokens in
the input.  
- (D) Self-attention results in contextual embeddings. 
- (E) Transformers seem to be more intuitive compared to LSTMs. 

<br><br><br><br>

```{admonition} Exercise 8.2: V's Solutions!
:class: tip, dropdown
- (A) True
- (B) False
- (C) True
- (D) True
- (E) True (for me)
```

<br><br><br><br>

## Multi-head attention

- Different words in a sentence can relate to each other in many different ways simultaneously. 
- Consider the sentence below. 
> The cat was scared because it didn't recognize me in my mask. 

Let's look at all the dependencies in this sentence. 

In [2]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")

In [3]:
doc = nlp("I gave my cat some food")
displacy.render(doc, style="dep")

- So a single attention layer usually is not enough to capture all different kinds of parallel relations between inputs. 
- Transformers address this issue with **multihead self-attention layers**.
- These self-attention layers are called **heads**.
- They are at the same depth of the model, operate in parallel, each with a different set of parameters. 
- The idea is that with these different sets of parameters, each head can learn different aspects of the relationships that exist among inputs.

\begin{equation}
\begin{split}
MultiHeadAttn(X) &= (\text{head}_1 \oplus \text{head}_2 \dots \oplus \text{head}_h)W^O\\
               Q &= XW_i^Q ; K = XW_i^K ; V = XW_i^V\\
               \text{head}_i &= SelfAttention(Q,K,V)
\end{split}
\end{equation}

![](img/multihead_attention.png)

<!-- <img src="img/multihead_attention.png" width="600" height="600"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Multi-head attention visualization
- Similar to RNNs you can stack self-attention layers or multihead self-attention layers on the top of each other.
- Let's look at this visualization which shows where the attention of different attention heads is going in multihead attention. 
    - [Multi-head attention interactive visualization](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb#scrollTo=OJKU36QAfqOC)

### (Optional) Let's try it out with PyTorch

In [4]:
from torch.nn import MultiheadAttention

In [5]:
embed_dim = 12
num_heads = 4
seqlen = 20

multi_attn = nn.MultiheadAttention(embed_dim, num_heads)

In [6]:
query = torch.rand(10, embed_dim)  # target_seq_len, query_embedding_dim
key = torch.rand(seqlen, embed_dim)  # source_seq_len, key_embedding_dim
value = torch.rand(seqlen, embed_dim)  # source_seq_len, value_embedding_dim

In [7]:
attn_output, attn_output_weights = multi_attn(query, key, value)

In [8]:
attn_output.shape

torch.Size([10, 12])

In [9]:
attn_output

tensor([[ 0.0234, -0.1684, -0.0636,  0.1402,  0.0948, -0.0758, -0.2251,  0.0719,
          0.2606,  0.1671, -0.1268, -0.2008],
        [ 0.0224, -0.1690, -0.0622,  0.1398,  0.0944, -0.0767, -0.2234,  0.0721,
          0.2599,  0.1671, -0.1263, -0.2007],
        [ 0.0206, -0.1691, -0.0622,  0.1391,  0.0945, -0.0769, -0.2237,  0.0728,
          0.2610,  0.1680, -0.1266, -0.2000],
        [ 0.0232, -0.1680, -0.0623,  0.1394,  0.0941, -0.0760, -0.2240,  0.0722,
          0.2594,  0.1672, -0.1267, -0.2005],
        [ 0.0225, -0.1687, -0.0631,  0.1392,  0.0945, -0.0760, -0.2242,  0.0730,
          0.2604,  0.1670, -0.1271, -0.2004],
        [ 0.0236, -0.1682, -0.0623,  0.1401,  0.0949, -0.0752, -0.2253,  0.0726,
          0.2608,  0.1682, -0.1266, -0.2019],
        [ 0.0224, -0.1696, -0.0640,  0.1400,  0.0953, -0.0768, -0.2245,  0.0726,
          0.2602,  0.1663, -0.1268, -0.2002],
        [ 0.0233, -0.1684, -0.0624,  0.1400,  0.0948, -0.0759, -0.2243,  0.0721,
          0.2604,  0.1680, -0.

In [10]:
attn_output_weights.shape

torch.Size([10, 20])

In [11]:
attn_output_weights.shape

torch.Size([10, 20])

<br><br>

### Transformers as language models

- Given a training corpus of plain text we want to train a model to predict the next word in a sequence with semi-supervised learning. 
- At each time step, given all the preceding words, the final transformer layer produces an output distribution over the entire vocabulary. 
- During training the probability assigned to the correct word is used to calculate the cross-entropy loss for each item in the sequence. 
- Similar to RNNs, the loss for the training sequence is the average cross-entropy loss over the entire sequence. 

![](img/transformer_language_model.png)

<!-- <img src="img/transformer_language_model.png" width="700" height="700"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### RNN as language models

- What's the difference between RNN-based language models vs. Transformers-based language model? 
    - The calculation of outputs and the losses at each time step was inherently serial in RNNs.
    - With transformers, each training item can be processed in parallel.

![](img/RNN_language_model.png)

<!-- <img src="img/RNN_language_model.png" width="700" height="700"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

### Contextual generation 

- A seed is provided and the model is asked to generate a possible completition to it. 
- Also called **autoregressive text completion**. 
- During the generation process, the model has direct access to the priming context as well as the generated tokens. 

![](img/transformer_autoregressive_text_generation.png)

<!-- <img src="img/transformer_autoregressive_text_generation.png" width="700" height="700"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

<br><br><br><br>

## Bidirectional transformer encoders

- Models such as [**BERT**](https://en.wikipedia.org/wiki/BERT_(language_model)) and its variant **RoBERTa** are **bidirectional transformer models**.  
- Remember the [sentence transformers](https://www.sbert.net/) you used in DSCI 563 lab1 to get sentence embeddings? They are based on **BERT**.  
- In the lab you're doing transfer learning using RoBERTa which was proposed in 2019. 
    - If you look at the `config_spacy.cfg`, you'll see the name of the model which is `roberta-base`.  
- What underlies these models? 

### Backward looking self-attention 

- We have seen backward looking self-attention. 
- Each output is computed using only information seen **earlier** in the context. 
- This is appropriate for language models. 

![](img/self_attention.png)

<!-- <img src="img/self_attention.png" width="700" height="700"> -->

#### Bidirectional self-attention

- Information flows in both directions in bidirectional self attention. 
- The model attends to all inputs, both before and after the current one.

![](img/bidirectional_self_attention.png)

<!-- <img src="img/bidirectional_self_attention.png" width="700" height="700"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

- All the computations are exactly the same as before. 
- The matrix below shows $q_i \cdot k_i$ comparisons. We do not set the values in the upper triangle to $\infty$ anymore. 

![](img/self_attention_calc_all.png)

<!-- <img src="img/self_attention_calc_all.png" width="400" height="400"> -->

The original bidirectional transformer encoder model (BERT) consisted of the following:

- A subword vocabulary of 30,000 tokens 
- Hidden layers of size 768 (If you recall the sentence embeddings from DSCI 563 lab 1 were 768 dimensional.)
- 12 layers of transformer blocks with 12 multihead attention layers each! 

The model has over 100M parameters. 

## Contextual embeddings 

- The methods like word2vec learned a single vector embedding for each unique word $w$ in the vocabulary. 
- By contrast, with contextual embeddings, such as those learned by popular methods such like BERT or GPT or their descendants, each word $w$ will be represented by a different vector each time it appears in a different context. 


## Transfer learning
- Pretraining is the process of learning some representation of input (words or sentences in NLP context) by processing large amounts of text.  
- Fine-tuning is the process of taking the representations from these pretrained models and further training the model often by adding a neural net classifier to perform some downstream task such as named entity recognition similar to what you are doing in the lab. 
- The intuition is that the pretrained phase learns a language model which in the process learns a rich representation of contextual word meaning enabling the model to be fine-tuned to the requirements of the downstream language understanding task.   
- This pretrain and fine-tune paradigm is called **transfer learning** in machine learning. 

![](img/BERT_sequence_labeling.png)

<!-- <img src="img/BERT_sequence_labeling.png" width="700" height="700"> -->

[Source](https://web.stanford.edu/~jurafsky/slp3/9.pdf)

There are many things related to transformers which we have not covered. You can look up the following if you want to know more. [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/) is an excellent resource. 
- Seq2seq models with attention 
- Masked language modeling 
- Contextual embeddings 
- ...

Transformers are not only for NLP. They have been successfully applied in many other domains often with state-of-the-art results. For example, 
- [Vision Transformers](https://arxiv.org/pdf/2010.11929.pdf)
- Bioinformatics: See [this](https://www.deepmind.com/blog/alphafold-a-solution-to-a-50-year-old-grand-challenge-in-biology) and [this](http://people.csail.mit.edu/tommi/papers/Ingraham_etal_neurips19.pdf).

<br><br><br><br>

## Resources
Attention-mechanisms and transformers are quite new. But there are many resources on transformers. I'm listing a few resources here. 

- [Attention is all you need](https://arxiv.org/pdf/1706.03762.pdf)
- [Transformers](https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html)
- [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [Transformers documentation](https://huggingface.co/transformers/index.html)
- [A funny video: I taught an AI to make pasta](https://www.youtube.com/watch?v=Y_NvR5dIaOY)
- [BERT](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)

<br><br><br><br>

## Summary and wrap up 

### Week 1 ✅

- Markov models, language models, text generation 

![](img/Markov_autocompletion.png)

<!-- <center> -->
<!-- <img src="img/Markov_autocompletion.png" height="800" width="800">  -->
<!-- </center>     -->
    

### Applications of Markov models

![](img/Markov_chain_applications.png)

<!-- <center> -->
<!-- <img src="img/Markov_chain_applications.png" width="500" height="500"> -->
<!-- </center>     -->
    

### Week 2 ✅

- Hidden Markov models, speech recognition, POS tagging

![](img/hmm_eks.gif)

<!-- <center> -->
<!-- <img src="img/hmm_eks.gif" height="800" width="800"> -->
<!-- </center> -->

    

### Week 3 ✅

- Topic modeling (Latent Dirichlet Allocation (LDA)), organizing documents 
- Introduction to Recurrent Neural Networks (RNNs)

![](img/TM_food_magazines.png)

<!-- <center> -->
<!-- <img src="img/TM_food_magazines.png" height="1000" width="1000">  -->
<!-- </center>     -->


### Week 4 ✅

- LSTMs, Transformers, Custom NER using transfer learning 


![](img/eva-accomplished.png)

### Final remarks 

That's all! I had fun teaching you this complex material. I very much appreciate your support, patience, and great questions ♥️!   

It has been a challenging year but we all tried to make the best out of it. I wish you every success in your job search. Stay in touch!

### Time for course evaluations

I would love to hear your thoughts on this course. When you get a chance, it'll be great if you fill in the evaluation survey for this course on [Canvas](https://canvas.ubc.ca/courses/83559/external_tools/4732). 

The evaluation closing date is: **April 29th, 2022**

<br><br><br><br>